## Benchmark for measuring performance of engineered features and models
Add station flow to benchmark_1

In [1]:
import os
if os.name=='nt':
    mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-8.1.0-posix-seh-rt_v6-rev0\\mingw64\\bin'
    os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate, cross_val_predict, StratifiedKFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from sys import getsizeof
import time
import gc

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
import pickle

def save_pickle(x, filename):
    with open(filename, 'wb') as handle:
        pickle.dump(x, handle, protocol=pickle.HIGHEST_PROTOCOL)

def read_pickle(filename):
    with open(filename, 'rb') as handle:
        x = pickle.load(handle)
    return x

In [3]:
from numba import jit

@jit
def mcc(tp, tn, fp, fn):
    sup = tp * tn - fp * fn
    inf = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    if inf==0:
        return 0
    else:
        return sup / np.sqrt(inf)

@jit
def eval_mcc(y_true, y_prob, show=False):
    idx = np.argsort(y_prob)
    y_true_sort = y_true[idx]
    n = y_true.shape[0]
    nump = 1.0 * np.sum(y_true) # number of positive
    numn = n - nump # number of negative
    tp = nump
    tn = 0.0
    fp = numn
    fn = 0.0
    best_mcc = 0.0
    best_id = -1
    prev_proba = -1
    best_proba = -1
    mccs = np.zeros(n)
    new_mcc = 0
    for i in range(n):
        # all items with idx < i are predicted negative while others are predicted positive
        # only evaluate mcc when probability changes
        proba = y_prob[idx[i]]
        if proba != prev_proba:
            prev_proba = proba
            new_mcc = mcc(tp, tn, fp, fn)
            if new_mcc >= best_mcc:
                best_mcc = new_mcc
                best_id = i
                best_proba = proba
        mccs[i] = new_mcc
        if y_true_sort[i] == 1:
            tp -= 1.0
            fn += 1.0
        else:
            fp -= 1.0
            tn += 1.0
    if show:
        y_pred = (y_prob >= best_proba).astype(int)
        score = matthews_corrcoef(y_true, y_pred)
        print(score, best_mcc)
        plt.plot(mccs)
        return best_proba, best_mcc, y_pred
    else:
        return best_mcc
    
def mcc_eval(y_prob, dtrain):
    y_true = dtrain.get_label()
    best_mcc = eval_mcc(y_true, y_prob)
    return 'MCC', -best_mcc

In [4]:
def cross_val_predict_rskf(clf, x_train, y_train, n_splits=3, 
                           n_repeats=2, random_state=42, verbose=False, early_stopping=None):
    '''
    Repeated stratified KFold CV, returns predictions for 
    each repeat and average score.
    n_repeats: repetitions of CV
    to disable erlay stopping, set early_stopping to None
    '''
    scores = []
    n_trees = []
    clfs = []
    running_time = []
    
    rskf = RepeatedStratifiedKFold(n_repeats=n_repeats, n_splits=n_splits, 
                                   random_state=0)
    np.random.seed(random_state)
    for n, (train_index, test_index) in enumerate(rskf.split(x_train, y_train)):
        print('Round {}'.format(n))
        start_time = time.time()
        x_train_tmp, x_test_tmp = x_train.iloc[train_index], x_train.iloc[test_index]
        y_train_tmp, y_test_tmp = y_train.iloc[train_index], y_train.iloc[test_index]

        clf.random_state = np.random.randint(10000000)
        
        if early_stopping is not None:
            clf.fit(x_train_tmp, y_train_tmp, 
                    eval_set=[(x_test_tmp, y_test_tmp)], 
                    eval_metric=mcc_eval, early_stopping_rounds=early_stopping,
                    verbose=verbose)
            scores.append(-clf.best_score)
            n_trees.append(clf.best_ntree_limit)
        else:
            clf.fit(x_train_tmp, y_train_tmp)
            scores.append(eval_mcc(y_test_tmp.values, clf.predict_proba(x_test_tmp)[:, 1]))
            n_trees.append(clf.n_estimators)
        
        clfs.append(clf)
        running_time.append(time.time()-start_time)
        print('Split {}, score = {:.3f}, best_ntree_limit = {}, total time = {:.3f} min'.format(n, scores[n], 
            n_trees[n], sum(running_time)/60))

    print('Score mean = {:.3f}, std = {:.3f}'.format(np.mean(scores), np.std(scores)))
    
    return clfs, scores, n_trees, running_time

In [5]:
def cross_val_predict_skf_rm(clf, x_train, y_train, n_splits=3, 
                           n_repeats=2, random_state=42, verbose=False, early_stopping=None):
    '''
    Stratified KFold CV with repeated models
    to disable erlay stopping, set early_stopping to None
    '''
    scores = []
    n_trees = []
    clfs = []
    running_time = []

    skf = StratifiedKFold(n_splits=n_splits, random_state=0, shuffle=True)
    np.random.seed(random_state)

    for m in range(n_repeats):
        print('Repeat {}'.format(m))
        for n, (train_index, test_index) in enumerate(skf.split(x_train, y_train)):
            start_time = time.time()
            x_train_tmp, x_test_tmp = x_train.iloc[train_index], x_train.iloc[test_index]
            y_train_tmp, y_test_tmp = y_train.iloc[train_index], y_train.iloc[test_index]

            clf.random_state = np.random.randint(10000000)
            # print(clf.random_state)

            if early_stopping is not None:
                clf.fit(x_train_tmp, y_train_tmp, 
                        eval_set=[(x_test_tmp, y_test_tmp)], 
                        eval_metric=mcc_eval, early_stopping_rounds=early_stopping,
                        verbose=verbose)
                scores.append(-clf.best_score)
                n_trees.append(clf.best_ntree_limit)
            else:
                clf.fit(x_train_tmp, y_train_tmp)
                scores.append(eval_mcc(y_test_tmp.values, clf.predict_proba(x_test_tmp)[:, 1]))
                n_trees.append(clf.n_estimators)
            
            clfs.append(clf)
            running_time.append(time.time() - start_time)
            print('Split {}, score = {:.3f}, n_best_trees = {}, total time = {:.3f} min'.format(n, 
                scores[m*n_repeats+n], n_trees[m*n_repeats+n], sum(running_time)/60))

    print('Score mean = {:.3f}, std = {:.3f}'.format(np.mean(scores), np.std(scores)))
    
    return clfs, scores, n_trees, running_time

In [6]:
# y_train = read_pickle('y_train.pickle')

# x_train = read_pickle('x_train_numeric_date_0.pickle')

In [7]:
important_features = pd.read_csv('benchmark_features/benchmark_1/important_numeric_features.csv', 
                                 index_col=0, header=None)
important_features = list(important_features.values.ravel())
important_features.extend(['Id', 'Response'])

In [8]:
x_train_numeric = pd.read_csv('data/train_numeric.csv.zip', index_col=0, usecols=important_features, dtype=np.float32)

y_train = x_train_numeric.Response
x_train_numeric.drop(['Response'], axis=1, inplace=True)

In [9]:
date_train = pd.read_csv('benchmark_features/benchmark_2/train_station_flow.csv.gz', index_col=0, header=None)

In [10]:
x_train = x_train_numeric.join(date_train)

columns = list(x_train.columns)
columns[-1] = 'station_flow'
x_train.columns = columns

### CV score based on stratified KFold with repeated models

In [11]:
n_estimators = 20
clf = XGBClassifier(max_depth=14, n_estimators=n_estimators, 
                    base_score=0.0058, n_jobs=20, colsample_bytree=0.6,
                    min_child_weight=5, subsample=0.9,  
                    reg_lambda=4, silent=False, learning_rate=0.03)

# clfs, scores, n_trees, running_time = cross_val_predict_rskf(clf, x_train_numeric, 
#    y_train, n_splits=3, n_repeats=2, random_state=42, verbose=False)

clfs, scores, n_trees, running_time = cross_val_predict_skf_rm(clf, x_train, 
    y_train, n_splits=5, n_repeats=3, random_state=4238457, early_stopping=None, verbose=False)
# save_pickle(clfs, 'clfs_benchmark_2_flow_analysis.pickle')

results = {'clfs': clfs, 'scores': scores, 'n_trees': n_trees, 'running_time': running_time}
save_pickle(results, 'benchmark_features/benchmark_2/results_benchmark_2_flow_analysis.pickle')

Repeat 0
Split 0, score = 0.222, n_best_trees = 20, total time = 1.610 min
Split 1, score = 0.226, n_best_trees = 20, total time = 3.201 min
Split 2, score = 0.223, n_best_trees = 20, total time = 4.817 min
Split 3, score = 0.216, n_best_trees = 20, total time = 6.407 min
Split 4, score = 0.219, n_best_trees = 20, total time = 8.051 min
Repeat 1
Split 0, score = 0.216, n_best_trees = 20, total time = 9.674 min
Split 1, score = 0.219, n_best_trees = 20, total time = 11.274 min
Split 2, score = 0.218, n_best_trees = 20, total time = 12.878 min
Split 3, score = 0.222, n_best_trees = 20, total time = 14.500 min
Split 4, score = 0.224, n_best_trees = 20, total time = 16.088 min
Repeat 2
Split 0, score = 0.222, n_best_trees = 20, total time = 17.710 min
Split 1, score = 0.224, n_best_trees = 20, total time = 19.305 min
Split 2, score = 0.217, n_best_trees = 20, total time = 20.903 min
Split 3, score = 0.221, n_best_trees = 20, total time = 22.480 min
Split 4, score = 0.213, n_best_trees = 20

### Calculate `n_trees` for training the whole dataset

In [12]:
results = read_pickle('benchmark_features/benchmark_2/results_benchmark_2_flow_analysis.pickle')
n_rounds = np.ceil(np.mean(results['n_trees'])*5/4).astype(np.int)
np.mean(results['scores'])

0.2197476668911593

### Train on the whole dataset

In [13]:
clf = XGBClassifier(max_depth=14, n_estimators=n_rounds, 
                    base_score=0.0058, n_jobs=-1, colsample_bytree=0.6,
                    min_child_weight=5, subsample=0.9,  
                    reg_lambda=4, silent=False, learning_rate=0.03)
clf.fit(x_train, y_train, 
        eval_set=[(x_train, y_train)], 
        eval_metric=mcc_eval, verbose=True)

[0]	validation_0-error:0.005811	validation_0-MCC:-0.171849
[1]	validation_0-error:0.005811	validation_0-MCC:-0.182925
[2]	validation_0-error:0.005811	validation_0-MCC:-0.189521
[3]	validation_0-error:0.005811	validation_0-MCC:-0.191187
[4]	validation_0-error:0.005811	validation_0-MCC:-0.197343
[5]	validation_0-error:0.005811	validation_0-MCC:-0.211479
[6]	validation_0-error:0.005811	validation_0-MCC:-0.220939
[7]	validation_0-error:0.005811	validation_0-MCC:-0.223311
[8]	validation_0-error:0.005811	validation_0-MCC:-0.224485
[9]	validation_0-error:0.005811	validation_0-MCC:-0.226756
[10]	validation_0-error:0.005811	validation_0-MCC:-0.22919
[11]	validation_0-error:0.005811	validation_0-MCC:-0.233556
[12]	validation_0-error:0.005811	validation_0-MCC:-0.235394
[13]	validation_0-error:0.005811	validation_0-MCC:-0.237844
[14]	validation_0-error:0.005811	validation_0-MCC:-0.240382
[15]	validation_0-error:0.005811	validation_0-MCC:-0.241101
[16]	validation_0-error:0.005811	validation_0-MCC:-

XGBClassifier(base_score=0.0058, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=0, learning_rate=0.03, max_delta_step=0,
       max_depth=14, min_child_weight=5, missing=None, n_estimators=25,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=4, scale_pos_weight=1,
       seed=None, silent=False, subsample=0.9)

In [14]:
y_train_pred = clf.predict_proba(x_train)[:, 1]

In [ ]:
thresholds = np.linspace(0.01, 0.99, 400)
mcc = np.array([matthews_corrcoef(y_train, y_train_pred>thr) for thr in thresholds])
plt.plot(thresholds, mcc)
best_threshold = thresholds[mcc.argmax()]
print(mcc.max())

In [ ]:
best_threshold

### Load test data and predict

In [ ]:
important_features = pd.read_csv('benchmark_features/benchmark_1/important_numeric_features.csv', 
                                 index_col=0, header=None).values.ravel().tolist()
important_features.append('Id')

x_test = pd.read_csv('data/test_numeric.csv.zip', index_col=0, usecols=important_features)

date_test = pd.read_csv('benchmark_features/benchmark_2/test_station_flow.csv.gz', index_col=0, header=None)

date_test.index.name = 'Id'
date_test.columns = ['station_flow']

x_test = x_test.join(date_test, on='Id')
x_test.columns = columns

In [ ]:
y_test_pred = clf.predict_proba(x_test)[:, 1]
y_test_pred_int = (y_test_pred>best_threshold).astype(int)

In [ ]:
sub = pd.read_csv("data/sample_submission.csv.zip", index_col=0)
sub["Response"] = y_test_pred_int
sub.to_csv("benchmark_features/benchmark_2/15-benchmark_2_flow_analysis_submission.csv.gz", compression="gzip")

public MCC = 0.22579, private MCC = 0.21225

CV MCC mean = 0.22643, std = 0.009

**Hyperparameter tuning is needed!**